In [ ]:
#default_exp imaml
#export
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import higher
import wandb

import pytorch_lightning as pl
from itertools import repeat

from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from unsupervised_meta_learning.pl_dataloaders import OmniglotDataModule
from unsupervised_meta_learning.hessian_free import HessianFree
from unsupervised_meta_learning.nn_utils import get_accuracy
from unsupervised_meta_learning.maml import ConvolutionalNeuralNetwork

import unsupervised_meta_learning.hypergrad as hg

In [ ]:
#export
class iMAML(pl.LightningModule):
    def __init__(self, model, meta_lr, inner_lr, inner_steps, cg_steps, reg_param, hg_mode='CG'):
        super().__init__()
        self.automatic_optimization = False
        self.model = model
        self.meta_lr = meta_lr
        self.inner_lr = inner_lr
        self.inner_steps = inner_steps
        self.cg_steps = cg_steps
        self.n_params = len(list(model.parameters()))
        self.reg_param = reg_param
        self.hg_mode = hg_mode
        self.T = 16
        self.K = 5
        
        self.fmodel = higher.monkeypatch(model, device=self.device, copy_initial_weights=True)
        self.inner_opt_cls = hg.GradientDescent
    
    def bias_reg_f(self, bias, params):
        return sum(
            [((b - p) ** 2).sum() for b, p in zip(bias, params)]
        )
    
    def train_loss_f(self, params, hparams):
        o = self.fmodel(self.tr_x, params=params)
        return F.cross_entropy(o, self.tr_y) + .5 + self.reg_param + self.bias_reg_f(hparams, params)
    
    def val_loss_f(self, params, hparams):
        o = self.fmodel(self.tst_x, params=params)
        val_loss = F.cross_entropy(o, self.tst_y) / self.batch_size
        self.val_loss = val_loss.item()
        pred = o.argmax(dim=1, keepdim=True)
        self.val_acc = pred.eq(self.tst_y.view_as(pred)).sum().item() / len(self.tst_y)
        return val_loss
    
    
    def get_inner_opt(self, train_loss, kwargs):
        return self.inner_opt_cls(train_loss, **kwargs)
    
    def inner_loop(self, hparams, params, optim, n_steps, log_interval, create_graph=False):
        params_history = [optim.get_opt_params(params)]

        for t in range(n_steps):
            params_history.append(optim(params_history[-1], hparams, create_graph=create_graph))
            self.log('loss', optim.curr_loss.item(), on_step=True, prog_bar=True, logger=True)

        return params_history
    
    def configure_optimizers(self):
        outer_opt = torch.optim.Adam(params=self.model.parameters(), lr=1e-3)
        return outer_opt
    
    @torch.enable_grad()
    def meta_learn(self, batch, batch_idx):
        meta_optimizer = self.optimizers()
        meta_optimizer = meta_optimizer.optimizer        
        tr_xs, tr_ys = batch["train"][0].to(self.device), batch["train"][1].to(self.device)
        tst_xs, tst_ys = batch["test"][0].to(self.device), batch["test"][1].to(self.device)
        
                
        self.batch_size = tr_xs.shape[0]
        val_loss, val_acc = torch.tensor(0., device=self.device), torch.tensor(0., device=self.device)
        
        inner_opt_kwargs = {'step_size': self.inner_lr}
        
        
        meta_optimizer.zero_grad()
        for t_idx, (tr_x, tr_y, tst_x, tst_y) in enumerate(zip(tr_xs, tr_ys, tst_xs, tst_ys)):
            self.tr_x = tr_x
            self.tr_y = tr_y

            self.tst_x = tst_x
            self.tst_y = tst_y
            inner_opt = self.get_inner_opt(self.train_loss_f, inner_opt_kwargs)
            
            params = [p.detach().clone().requires_grad_(True) for p in self.model.parameters()]
            last_param = self.inner_loop(self.model.parameters(), params, inner_opt, self.T, log_interval=None)[-1]
            
            if self.hg_mode == 'CG':
                # This is the approximation used in the paper CG stands for conjugate gradient
                cg_fp_map = hg.GradientDescent(loss_f=self.train_loss_f, step_size=1.)
                hg.CG(last_param, list(self.model.parameters()), K=self.K, fp_map=cg_fp_map, outer_loss=self.val_loss_f)
            elif self.hg_mode == 'fixed_point':
                hg.fixed_point(last_param, list(seld.model.parameters()), K=self.K, fp_map=inner_opt,
                               outer_loss=self.val_loss_f)
            
            val_loss += self.val_loss
            val_acc += self.val_acc / self.batch_size

        meta_optimizer.step()
        return val_loss, val_acc
    
    def training_step(self, batch, batch_idx):
        train_loss, train_acc = self.meta_learn(batch, batch_idx)
        self.log_dict({
            'tr_accuracy': train_acc.item(),
            'tr_loss': train_loss.item()
        }, prog_bar=True, logger=True)
        return {'tr_loss': train_loss.item(), 'tr_acc': train_acc.item()}TensorBoard
        
    
    def validation_step(self, batch, batch_idx):
        val_loss, val_acc = self.meta_learn(batch, batch_idx)
        
        self.log_dict({
            'val_loss': val_loss.item(),
            'val_accuracy': val_acc.item()
        })
        return val_loss.item()
    
    def test_step(self, batch, batch_idx):
        test_loss, test_acc = self.meta_learn(batch, batch_idx)
        self.log_dict({
            'test_loss': test_loss.item(),
            'test_accuracy': test_acc.item()
        })
        return test_loss.item()

In [ ]:
dm = OmniglotDataModule(
        "data",
        shots=1,
        ways=5,
        shuffle_ds=True,
        test_shots=16,
        meta_train=True,
        download=True,
        batch_size=16,
        shuffle=True,
        num_workers=8,
)

In [ ]:
model = iMAML(model=ConvolutionalNeuralNetwork(1, 5, hidden_size=64), meta_lr=1e-3, inner_lr=1e-2, reg_param=2, inner_steps=1, cg_steps=5, )

In [ ]:
logger = WandbLogger(
    project='iMAML',
    config={
        'batch_size': 16,
        'steps': 100,
        'dataset': "omniglot",
        'T': 16,
        'K': 5,
        'val/test': 'enabled'
    }
)
trainer = pl.Trainer(
        profiler='simple',
        max_steps=28,
        val_check_interval=25,
        limit_train_batches=26,
        limit_val_batches=2,
        limit_test_batches=2,
        fast_dev_run=False,
        gpus=1,
        logger=logger,
        log_every_n_steps=1,
        flush_logs_every_n_steps=1
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: p0int (use `wandb login --relogin` to force relogin)



  | Name   | Type                                 | Params
----------------------------------------------------------------
0 | model  | ConvolutionalNeuralNetwork           | 112 K 
1 | fmodel | FunctionalConvolutionalNeuralNetwork | 112 K 
----------------------------------------------------------------
224 K     Trainable params
0         Non-trainable params
224 K     Total params
0.898     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  65.453         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  26.36          	|2              	|  52.719         	|  80.545         	|
run_training_batch                 	|  1.6216         	|28             	|  45.404         	|  69.368         	|
model_forward                      	|  1.6214         	|28             	|  45.399         	|  69.362         	|
training_step                      	|  1.6212         	|28             	|  45.395         	|  69.355         	|
evaluation_step_and_end            

In [ ]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/lib/python3.9/site-packages/torchvision/transforms/functional.py:942: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ojass/anaconda3/envs/jax/l

In [ ]:
wandb.finish()

loss,2.5236
tr_accuracy,0.84531
tr_loss,0.5584
epoch,1
trainer/global_step,27
_runtime,61
_timestamp,1623095306
_step,30
loss_step/epoch_0,2.53318
loss_epoch,2.52925
val_loss,0.67879


loss,▇▄▅▆▇▆▆▇▆▅▄▅▄▄▄▅▅▃▅▃▄█▃▃▃▁▂▁
tr_accuracy,▂▂▁▃▄▅▆▅▃▄▅▇▄▇▆▆▆▅▆█▇██▇▆█▇▇
tr_loss,███▆▆▅▅▅▆▅▅▃▅▃▃▄▂▄▃▂▂▁▂▃▃▁▂▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██
trainer/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▁▁▇▇██
_runtime,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
loss_step/epoch_0,▁█
loss_epoch,▁
val_loss,▁
